In [1]:
!pip install -U hopsworks

You should consider upgrading via the 'C:\Users\ayber\PycharmProjects\Scalable_ML_Project_AQ\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install dataframe_image

You should consider upgrading via the 'C:\Users\ayber\PycharmProjects\Scalable_ML_Project_AQ\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
!pip install -q streamlit

You should consider upgrading via the 'C:\Users\ayber\PycharmProjects\Scalable_ML_Project_AQ\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [4]:
!pip install --upgrade protobuf

  Using cached protobuf-4.21.12-cp39-cp39-win_amd64.whl (527 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.12.0 requires protobuf<4,>=3.12, but you have protobuf 4.21.12 which is incompatible.
You should consider upgrading via the 'C:\Users\ayber\PycharmProjects\Scalable_ML_Project_AQ\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
!pip install streamlit_folium

  Using cached protobuf-3.20.3-cp39-cp39-win_amd64.whl (904 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.12
    Uninstalling protobuf-4.21.12:
      Successfully uninstalled protobuf-4.21.12


You should consider upgrading via the 'C:\Users\ayber\PycharmProjects\Scalable_ML_Project_AQ\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
!pip install pyngrok

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19793 sha256=8744185c15b53e1d4ccc1d7d9d63965faf0a9b8bfc150891cdcdacfa78f1d4b5
  Stored in directory: c:\users\ayber\appdata\local\pip\cache\wheels\f6\89\59\49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok


You should consider upgrading via the 'C:\Users\ayber\PycharmProjects\Scalable_ML_Project_AQ\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
!pip install python-dotenv

You should consider upgrading via the 'C:\Users\ayber\PycharmProjects\Scalable_ML_Project_AQ\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [8]:
from pyngrok import ngrok

In [9]:
ngrok.set_auth_token("2Jzz5UrcRRGTHRB1KeKe4M38bwG_mDhqRzgG61co9WY6dMVG")

In [10]:
import hopsworks
project = hopsworks.login()


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/5330


In [11]:
!git clone 'https://github.com/reyrobs/Scalable_ML_Project_AQ.git'

fatal: destination path 'Scalable_ML_Project_AQ' already exists and is not an empty directory.


In [11]:
!pip install sklearn

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=f02c13f66a2d6809f0b23709f0bf80040a483ca420118f0cb1d3584eb58a8bda
  Stored in directory: c:\users\ayber\appdata\local\pip\cache\wheels\f8\e0\3d\9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built sklearn


You should consider upgrading via the 'C:\Users\ayber\PycharmProjects\Scalable_ML_Project_AQ\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [12]:
%%writefile app.py
import streamlit as st
import hopsworks
import os
import joblib
import pandas as pd
import dataframe_image as dfi
import streamlit as st
import joblib
import numpy as np
import folium
from streamlit_folium import st_folium, folium_static
import json
import time
from datetime import timedelta, datetime
from branca.element import Figure
from urllib.request import urlopen
import joblib
from API_call_visual_crossing import *
from API_call_AQI  import *
#----------------------------------HEADER + TITLE
def fancy_header(text, font_size=24):
    res = f'<span style="color:#ff5f27; font-size: {font_size}px;">{text}</span>'
    st.markdown(res, unsafe_allow_html=True )
st.title('⛅️Air Quality Prediction Project🌩')

progress_bar = st.sidebar.header('⚙️ Working Progress')
progress_bar = st.sidebar.progress(0)
st.write(36 * "-")

#------------------------------------  Getting Data

fancy_header('\n☁️ Getting data for predict ...')
city = 'Paris'
date = ''


WEATHER_API_KEY ="V7VHELUHHCY7JPQ8332XPR8A9"# = os.getenv('WEATHER_API_KEY')

df_weather = get_weather_df(city, date, WEATHER_API_KEY)
next_7_days = df_weather["date"]
data_7_days = df_weather.drop('date', axis=1)


model = joblib.load(urlopen('https://github.com/reyrobs/Scalable_ML_Project_AQ/raw/main/aqi_model.pkl'))
progress_bar.progress(20)
st.write(36 * "-")

#------------------------------------ PREDICTING
fancy_header('\n ✔️Predicting...')

pred_7_days = model.predict(data_7_days)

df = pd.DataFrame(data=pred_7_days, index=next_7_days, columns=[f"AQI Predictions for the next 7 days"], dtype=int)

progress_bar.progress(60)
#----------------------------------

st.write(36 * "-")
fancy_header(f"🗺 Processing the map...")

fig = Figure(width=550,height=350)

my_map = folium.Map(location=[58, 20], zoom_start=3.71)
fig.add_child(my_map)
folium.TileLayer('Stamen Terrain').add_to(my_map)
folium.TileLayer('Stamen Toner').add_to(my_map)
folium.TileLayer('Stamen Water Color').add_to(my_map)
folium.TileLayer('cartodbpositron').add_to(my_map)
folium.TileLayer('cartodbdark_matter').add_to(my_map)
folium.LayerControl().add_to(my_map)

#Determining Paris
coords = [48.862, 2.346]
#-----
city = "Pris"
country="France"
text="AQI "

#-----
pred_7_days = np.asarray(pred_7_days, dtype = 'int')
#pred_7_days.round(decimals=1, out= pred_7_days)
text +=str((pred_7_days[0]))
#-----

folium.Marker(
        location=coords, popup=text, tooltip=f"<strong>{city}</strong>"
    ).add_to(my_map)


# call to render Folium map in Streamlit
folium_static(my_map)
progress_bar.progress(80)
st.sidebar.write("-" * 36)


st.sidebar.write(df)
progress_bar.progress(100)
st.button("Re-run")


Writing app.py


In [14]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

OSError: Background processes not supported.